<a href="https://colab.research.google.com/github/kdj0712/study_AIs/blob/main/docs/quests/NLPs/classification_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 사전작업

In [ ]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install python-mecab-ko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 39.6 MB/s eta 0:00:00


In [5]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 49.5 MB/s eta 0:00:00


## 모델 대상화 데이터 준비
- DDA,EDA 확인

In [6]:
import pandas as pd

In [7]:
df_train =  pd.read_csv('/content/drive/MyDrive/data_analytics/NLP(Natural Language Processing)/Quest/train_data.csv')
df_topic = pd.read_csv('/content/drive/MyDrive/data_analytics/NLP(Natural Language Processing)/Quest/topic_dict.csv')
df_test = pd.read_csv('/content/drive/MyDrive/data_analytics/NLP(Natural Language Processing)/Quest/test_data.csv')

In [8]:
df_topic

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


In [9]:
df_train[:2]

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4


In [10]:
df_test[:2]

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사


In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45654 entries, 0 to 45653
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      45654 non-null  int64 
 1   title      45654 non-null  object
 2   topic_idx  45654 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ MB


In [12]:
df_topic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   topic      7 non-null      object
 1   topic_idx  7 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 240.0+ bytes


In [13]:
df_topic

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9131 entries, 0 to 9130
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   9131 non-null   int64 
 1   title   9131 non-null   object
dtypes: int64(1), object(1)
memory usage: 142.8+ KB


### 데이터 전처리
- trainset, testset 병합
- tokenizing : 머신에게 맞게 Vector

In [15]:
raw_concat = pd.concat([df_train,df_test])
raw_concat.shape, df_train.shape, df_test.shape

((54785, 3), (45654, 3), (9131, 2))

In [16]:
# 숫자 삭제
raw_concat['title_preprocess'] = raw_concat['title'].str.replace('[0-9]','',regex=True)

In [17]:
# 영어 대문자 -> 소문자로 변환
raw_concat['title_preprocess'] = raw_concat['title_preprocess'].str.lower()

In [18]:
raw_concat['title_preprocess'].str.findall('[A-Z]').value_counts()

[]    54785
Name: title_preprocess, dtype: int64

In [19]:
from konlpy.tag import Okt
okt = Okt()

In [20]:
# 조사, 어미, 구두점 등 제거
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word[0])

    return " ".join(clean_text)

In [21]:
raw_concat['title_okt_clean'] = raw_concat['title_preprocess'].apply(okt_clean)

In [22]:
raw_concat[:5]

,index,title,topic_idx,title_preprocess,title_okt_clean
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4.0,인천→핀란드 항공기 결항…휴가철 여행객 분통,인천 → 핀란드 항공기 결항 휴가 철 여행객 분통
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4.0,실리콘밸리 넘어서겠다…구글 조원 들여 美전역 거점화,실리콘밸리 넘어서다 구글 조원 들이다 美 전역 거점 화
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4.0,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추다 것
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4.0,nyt 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,nyt 클린턴 측근 韓 기업 특수 관계 조명 공과 사 맞다 물리다 종합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4.0,시진핑 트럼프에 중미 무역협상 조속 타결 희망,시진핑 트럼프 중미 무역 협상 조속 타결 희망


In [23]:
# tokeninzing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None)
# tfidfVectorizer = TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [24]:
tfidfVectorizer.fit(raw_concat['title_okt_clean'])

TfidfVectorizer()

In [25]:
# trainset, testset 분리
trainset = raw_concat[raw_concat['topic_idx'].notnull()] # train
testset = raw_concat[raw_concat['topic_idx'].isnull()] # test
trainset.shape, testset.shape

((45654, 5), (9131, 5))

In [26]:
from mecab import MeCab
mecab = MeCab()

### 모델

In [27]:
train_target = trainset['topic_idx']
train_feature = tfidfVectorizer.transform(trainset['title_okt_clean']) # 머신 학습 위한 선작업

In [28]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier = RandomForestClassifier(n_jobs=-1)

In [29]:
randomForestClassifier.fit(X=train_feature, y=train_target)

RandomForestClassifier(n_jobs=-1)

In [55]:
import pickle
with open('ITnews.pkl','wb') as pickle_file :
  pickle.dump(obj=randomForestClassifier.fit(X=train_feature, y=train_target), file=pickle_file)

## 모델 적용 데이터

### read excel ifle with sheets

In [30]:
df_ITNews = pd.read_excel('/content/drive/MyDrive/data_analytics/CSV파일/IT News.xlsx',sheet_name=None)

In [31]:
df_sheets = []
for sheet_name,df in df_ITNews.items():
  df_sheets.append(df)
  print(f'{sheet_name},{type(df)}')
  pass
len(df_sheets)

2024-01-31,<class 'pandas.core.frame.DataFrame'>
2024-01-30,<class 'pandas.core.frame.DataFrame'>
2024-01-29,<class 'pandas.core.frame.DataFrame'>
2024-01-26,<class 'pandas.core.frame.DataFrame'>
2024-01-25,<class 'pandas.core.frame.DataFrame'>
2024-01-24,<class 'pandas.core.frame.DataFrame'>
2024-01-23,<class 'pandas.core.frame.DataFrame'>
2024-01-22,<class 'pandas.core.frame.DataFrame'>
2024-1-19,<class 'pandas.core.frame.DataFrame'>
2024-1-18,<class 'pandas.core.frame.DataFrame'>
2024-1-17,<class 'pandas.core.frame.DataFrame'>
2024-1-16,<class 'pandas.core.frame.DataFrame'>
2024-1-15,<class 'pandas.core.frame.DataFrame'>
2024-1-12,<class 'pandas.core.frame.DataFrame'>
2024-1-11,<class 'pandas.core.frame.DataFrame'>
2024-1-10,<class 'pandas.core.frame.DataFrame'>
2024-1-9,<class 'pandas.core.frame.DataFrame'>
2024-1-8,<class 'pandas.core.frame.DataFrame'>
2024-1-5,<class 'pandas.core.frame.DataFrame'>
2024-1-4,<class 'pandas.core.frame.DataFrame'>
2024-01-03,<class 'pandas.core.frame

45

In [32]:
df_sheets[0][10:11]

,이름,선호도 투표,뉴스명,문장 요약,한 줄평
10,김덕재,NaN,"음악가 71% ""AI로 생계 잃을 것"" 우려…작사·작곡에는 활용",프랑스·독일의 음악가 열에 일곱은 생성형 인공지능(AI)으로 생계에 위협을 느낀다는...,AI의 발전도에 따라 산업의 방향성도 변화하고 있다


In [34]:
df_sheets_iloc = []
for x in range(len(df_sheets)):
  df_sheets_iloc.append(df_sheets[x].iloc[10][3])
  print(f'{sheet_name},{type(df)}')
  pass

df_sheets_iloc = pd.DataFrame(df_sheets_iloc, columns=['summary'])
len(df_sheets_iloc)

2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'pandas.core.frame.DataFrame'>
2023-12-13,<class 'p

45

In [44]:
df_ITNews_sheets = pd.DataFrame(df_sheets_iloc).dropna()
df_ITNews_sheets

,summary
0,프랑스·독일의 음악가 열에 일곱은 생성형 인공지능(AI)으로 생계에 위협을 느낀다는...
1,이에 대해 오픈 AI는 NYT가 악의적인 프롬프트 조작으로 일종의 버그 현상인 ‘역...
3,당연히 은행 직원일 것으로 예상했다. 하지만 전화를 걸어온 것은 사람이 아니라 인공...
5,"""FSD 베타12는 수 백만 개의 영상 클립으로 훈련된 단일 종단간 신경망으로 업..."
7,스캐터랩은 국내에서 챗GPT 열풍이 불기 전 생성형 AI 기술을 접목해 눈길을 끌었다
18,로봇이 주변 환경을 인식하고 스스로의 행동을 분석해 작업을 개선하는 대규모 인공지능...
20,"데이터 수집부터 정제, 가공, 평가까지 모든 프로세스를 제공하는 기술이다. 신제품 ..."
21,브로드컴은 새로운 제품 라인업과 구독 라이선스를 발표하면서 기존보다 기본 가격을 5...
22,"골드만삭스, 2035년까지 3억개 일자리 사라질 것\n'인력 감축' IT업계는 비핵..."
23,한동안 경쟁적으로 일어났던 기업 내 인공지능(AI) 도입이 지지부진한 상태에 머무르...


In [ ]:
# # 조사, 어미, 구두점 등 제거
# def okt_clean(text):
#     clean_texts = []
#     for word in okt.pos(text, stem=True):
#         if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
#             clean_texts.append(word[0])

#     return " ".join(clean_texts)

In [52]:
import re
def regex_df(df, column, patterns, repl):
  df['new'] = df[column]
  for pattern in patterns:
    df['new'] = df['new'].str.replace(pattern,repl)
  return df

patterns = '(으로 |에서의 |있다|에 |로 |은 |는 |이 |가 |과 |되고 |등 |의 |를 |을 |하며|며 |데 |\.|\,|\'|\n)'
repl = ' '
df_ITNews_sheets = regex_df(df_ITNews_sheets,'summary',patterns,repl)
print(df_ITNews_sheets)

                                              summary  \
0   프랑스·독일의 음악가 열에 일곱은 생성형 인공지능(AI)으로 생계에 위협을 느낀다는...   
1   이에 대해 오픈 AI는 NYT가 악의적인 프롬프트 조작으로 일종의 버그 현상인 ‘역...   
3   당연히 은행 직원일 것으로 예상했다. 하지만 전화를 걸어온 것은 사람이 아니라 인공...   
5    "FSD 베타12는 수 백만 개의 영상 클립으로 훈련된 단일 종단간 신경망으로 업...   
7     스캐터랩은 국내에서 챗GPT 열풍이 불기 전 생성형 AI 기술을 접목해 눈길을 끌었다   
18  로봇이 주변 환경을 인식하고 스스로의 행동을 분석해 작업을 개선하는 대규모 인공지능...   
20  데이터 수집부터 정제, 가공, 평가까지 모든 프로세스를 제공하는 기술이다. 신제품 ...   
21  브로드컴은 새로운 제품 라인업과 구독 라이선스를 발표하면서 기존보다 기본 가격을 5...   
22  골드만삭스, 2035년까지 3억개 일자리 사라질 것\n'인력 감축' IT업계는 비핵...   
23  한동안 경쟁적으로 일어났던 기업 내 인공지능(AI) 도입이 지지부진한 상태에 머무르...   
24  오픈AI는 북한의 해킹 조직 김수키가 대규모 사이버 공격에 인공지능(AI) 서비스를...   
25                       항저우방송국 실제 앵커 본딴 AI 앵커가 뉴스 진행   
26  2014년 설립된 국내 유니콘(기업가치 1조원 이상) 스타트업 '하이퍼커넥트'는 글...   
27  전 세계 주요 기업들이 생성 인공지능(AI)에 이어 휴머노이드 로봇 기술 확보에 힘...   
28  아마존이 구글의 쿠키 삭제 조치에 맞서 온라인 광고 타겟팅을 위한 고객 데이터를 확...   
29  “기업에서 인공지능(AI) 서비스를 구축하더라도 이를 받쳐줄 인프라가 없다면 충분한...   
30  중국 사지마비환자, 뇌에 칩 이식 통해 동작 실현

<ipython-input-52-3df6eb2a0e04>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['new'] = df['new'].str.replace(pattern,repl)


In [53]:
df_ITNews_sheets['new']

0     프랑스·독일  음악  열  일곱  생성형 인공지능 AI    생계  위협  느낀  ...
1        대해 오픈 AI  NYT  악 적인 프롬프트 조작   일종  버그 현상인 ‘역...
3     당연히  행 직원일 것   예상했    지만 전화  걸어온 것  사람  아니라 인공...
5      "FSD 베타12  수 백만 개  영상 클립   훈련된 단일 종단간 신경망   업...
7       스캐터랩  국내   챗GPT 열풍  불기 전 생성형 AI 기술  접목해 눈길  끌었 
18     봇  주변 환경  인식   스스   행동  분석해 작업  개선   대규모 인공지능...
20      터 수집부터 정제   공  평 까지 모든 프 세스  제공   기술    신제품 ...
21    브 드컴  새 운 제품 라인업  구독 라 선스  발표 면  기존보  기본  격  5...
22    골드만삭스  2035년까지 3억개 일자리 사라질 것  인력 감축  IT업계  비핵심...
23    한동안 경쟁적   일어났던 기업 내 인공지능 AI  도입  지지부진한 상태  머무르...
24    오픈AI  북한  해킹 조직 김수키  대규모 사 버 공격  인공지능 AI   비스 ...
25                         항저우방송국 실제 앵커 본딴 AI 앵커  뉴스 진행
26    2014년 설립된 국내 유니콘 기업 치 1조원  상  스타트업    퍼커넥트   글...
27    전 세계 주요 기업들  생성 인공지능 AI    어 휴머노 드  봇 기술 확보  힘...
28    아마존  구글  쿠키 삭제 조치  맞  온라인 광  타겟팅  위한  객   터  확...
29    “기업   인공지능 AI   비스  구축 더라도    받쳐줄 인프라  없 면 충분한...
30    중국 사지마비환자  뇌  칩  식 통해 동작 실현 " 신경 전자 기회   식  마비...
34    초거대AI추진협 회  건 문 제출 후 ‘저작권  슈 공동 대응 제안문  칭 

In [ ]:
# df_ITNews_sheets['new'] = df_ITNews_sheets[0].apply(okt_clean)

In [60]:
df_ITNews_sheets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 0 to 44
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   summary  22 non-null     object
 1   new      22 non-null     object
 2   okt_new  22 non-null     object
dtypes: object(3)
memory usage: 704.0+ bytes


In [61]:
df_ITNews_sheets['okt_new']

0     프랑스 독일 음악 열 일곱 생 성형 인공 지능 AI 생계 위협 느끼다 여론조사 결 ...
1     대해 오픈 AI NYT 악 적 프롬 프트 조작 일종 버그 현상 ‘ 역류 regurg...
3     당연하다 행 직원 일 것 예상 하다 전화 걸어오다 것 사람 아니다 인공 지능 AI ...
5     FSD 베타 12 수 백만 개 영상 클립 훈련 되다 단일 종단 간 신경망 업그레 드...
7          스캐터랩 국내 챗 GPT 열풍 불기 전 생 성형 AI 기술 접목 하다 눈길 끌다
18    봇 주변 환경 인식 스스 행동 분석 하다 작업 개선 대규모 인공 지능 AI 모델 기...
20    터 수집 정제 공 평 모든 프 세스 제공 기술 신제품 공개 함께 스마트 모빌리티 스...
21    브 드컴 새 운 제품 라인업 구독 선스 발표 면 기존 보 기본 격 50% 인 하다 ...
22    골드만삭스 2035년 까지 3억 개 일자리 사라지다 것 인력 감축 IT 업계 비 핵...
23    한동안 경쟁 적 일어나다 기업 내 인공 지능 AI 도입 지지 부진하다 상태 머무르다...
24    오픈 AI 북한 해킹 조직 김수 키 대규모 사 버 공격 인공 지능 AI 비스 사용 ...
25                        항저우 방송국 실제 앵커 본 딴 AI 앵커 뉴스 진행
26    2014년 설립 되다 국내 유니콘 기업 치 1조원 상 스타트업 퍼 커넥트 글 벌 영...
27        전 세계 주요 기업 들 생 성 인공 지능 AI 휴 멀다 드 봇 기술 확보 힘 쏟다
28    아마존 구글 쿠키 삭제 조치 맞다 온라인 광 타겟 팅 위 객 터 확보 자 본격 적 ...
29    “ 기업 인공 지능 AI 비스 구축 더라도 받치다 인프라 없다 면 충분하다 효 기대...
30    중국 사지 마비 환자 뇌 칩 식 통해 동작 실현 신경 전자 기회 식 마비 환자 획기...
34    초 거대 AI 추진 협 회 건 문 제출 후 ‘ 저작권 슈 공동 대응 제안 문

In [58]:
df_ITNews_sheets['okt_new'] = df_ITNews_sheets['new'].apply(okt_clean)
for_token = df_ITNews_sheets['okt_new']

In [62]:
news_token = tfidfVectorizer.transform(df_ITNews_sheets['okt_new'])
news_token

<22x27288 sparse matrix of type '<class 'numpy.float64'>'
	with 328 stored elements in Compressed Sparse Row format>

In [64]:
token_array = news_token.toarray()

In [66]:
import numpy as np
np.where(news_token.toarray() > 0)

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,
         7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11,
        11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15,
        15, 

In [67]:
file_path = '/content/ITnews.pkl'
with open(file_path, 'rb') as pickle_file:
  loaded_model = pickle.load(pickle_file)

In [70]:
loaded_model.predict(news_token.toarray())

array([0., 0., 0., 2., 0., 0., 0., 2., 0., 0., 0., 0., 1., 0., 0., 0., 4.,
       0., 0., 2., 0., 0.])

In [71]:
df_topic

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


In [72]:
df_ITNews_sheets.iloc[16]

summary    중국 사지마비환자, 뇌에 칩 이식 통해 동작 실현\n"'신경 전자 기회' 이식, 마...
new        중국 사지마비환자  뇌  칩  식 통해 동작 실현 " 신경 전자 기회   식  마비...
okt_new    중국 사지 마비 환자 뇌 칩 식 통해 동작 실현 신경 전자 기회 식 마비 환자 획기...
Name: 30, dtype: object